In [ ]:
from time import sleep
import requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

In [ ]:
url1 = "https://www.amazon.in/Women-Watches/b/ref=nav_shopall_sbc_wfashion_watches?ie=UTF8&node=2563505031"
url2 = "https://www.amazon.in/s/ref=lp_2563505031_pg_2?rh=n%3A1350387031%2Cn%3A%211350388031%2Cn%3A2563505031&page=2&ie=UTF8&qid=1532405415"
url3 = "https://www.amazon.in/s/ref=sr_pg_3?rh=n%3A1350387031%2Cn%3A%211350388031%2Cn%3A2563505031&page=3&ie=UTF8&qid=1532405541"
url4 = "https://www.amazon.in/s/ref=sr_pg_4?rh=n%3A1350387031%2Cn%3A%211350388031%2Cn%3A2563505031&page=4&ie=UTF8&qid=1532405568"
url5 = "https://www.amazon.in/s/ref=sr_pg_5?rh=n%3A1350387031%2Cn%3A%211350388031%2Cn%3A2563505031&page=5&ie=UTF8&qid=1532405608"
url6 = "https://www.amazon.in/s/ref=sr_pg_6?rh=n%3A1350387031%2Cn%3A%211350388031%2Cn%3A2563505031&page=6&ie=UTF8&qid=1532405667"

In [ ]:
page_number = 1

In [ ]:
page_url = "https://www.amazon.in/s/ref=sr_pg_{}?rh=n%3A1350387031%2Cn%3A%211350388031%2Cn%3A2563505031&page={}".format(page_number, page_number)

In [ ]:
page_url

In [ ]:
html = urlopen(page_url).read()
soup = BeautifulSoup(html, "html.parser")

In [ ]:
links = []

In [ ]:
for li in soup.find_all("li", {"class" : "s-result-item"}):
    links.append(li.a['href'])

In [ ]:
links[:5]

In [ ]:
def get_all_product_links_in_a_page(page_url):
    html = urlopen(page_url).read()
    soup = BeautifulSoup(html, "html.parser")
    
    links = [li.a['href'] for li in soup.find_all("li", {"class" : "s-result-item"})]
    return links

In [ ]:
links = get_all_product_links_in_a_page(page_url)

In [ ]:
link = links[2]

In [ ]:
link

In [ ]:
"product-reviews".join(link.split("dp"))

In [ ]:
page_number = 1

In [ ]:
reviews_url = "product-reviews".join(link.split("dp")) + "/ref=cm_cr_arp_d_paging_btm_{}?pageNumber={}".format(page_number, page_number)

In [ ]:
reviews_url

In [ ]:
html = urlopen(reviews_url).read()
soup = BeautifulSoup(html, "html.parser")

In [ ]:
reviews_product = []

In [ ]:
for div in soup.find_all("div", {"id" : "cm_cr-review_list"}):
    for _div in div.find_all("div", {"data-hook" : "review"}):
        for a in _div.find_all("a", {"data-hook" : "review-title"}):
            title = a.text
        for i in _div.find_all("i", {"data-hook" : "review-star-rating"}):
            rating = i.text
            rating = rating.split(".")[0]
        for span in _div.find_all("span", {"data-hook" : "review-body"}):
            review = (span.text)
        reviews_product.append([title, rating, review])

In [ ]:
reviews_product[0]

In [ ]:
columns = ["title", "rating", "review"]

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(data=reviews_product, columns=columns)

In [ ]:
df.head()

# helper functions

In [ ]:
def get_all_product_links_in_a_page(page_url):
    html = urlopen(page_url).read()
    soup = BeautifulSoup(html, "html.parser")
    
    links = [li.a['href'] for li in soup.find_all("li", {"class" : "s-result-item"})]
    return links

In [ ]:
def find_all_reviews_in_a_page(url):
    html = urlopen(url).read()
    soup = BeautifulSoup(html, "html.parser")
    
    reviews = []
    
    for div in soup.find_all("div", {"id" : "cm_cr-review_list"}):
        for _div in div:
            try:
                _id = _div["id"]
                review = (_div.find("span", {"data-hook" : "review-body"})).text
                reviews.append(review)
            except:
                pass
            
    return reviews

In [ ]:
def find_all_reviews_of_a_product(product_link):
    reviews = []
    
    for i in range(1, 100, 1):
        page_number = str(i)
        reviews_url = "product-reviews".join(product_link.split("dp")) + "/ref=cm_cr_arp_d_paging_btm_" + page_number + "?pageNumber=" + page_number
       
        try:
            _reviews = find_all_reviews_in_a_page(reviews_url)
        except:
            sleep(5)
            _reviews = find_all_reviews_in_a_page(reviews_url)
            
        if len(_reviews) > 0:
            reviews += _reviews
        else:
            break
            
    return reviews

In [ ]:
def get_all_reviews_of_a_category():
    url1 = "https://www.amazon.in/s/ref=sr_pg_"
    url2 = "?rh=n%3A1350387031%2Cn%3A%211350388031%2Cn%3A2563505031%2Cp_n_pct-off-with-tax%3A30-%2Cp_98%3A10440597031&page="
    url3 = "&bbn=2563505031&ie=UTF8&qid=1532002016&lo=watches"
    
    reviews = []
    
    for products_page in range(1, 1000, 1):
        page_number = str(products_page)
        page_url_of_products = url1 + page_number + url2 + page_number + url3
        
        try:
            links = get_all_product_links_in_a_page(page_url_of_products)
        except:
            sleep(5)
            links = get_all_product_links_in_a_page(page_url_of_products)
        
        print(products_page, len(links))
            
        if len(links) > 0:
            for product_link in links:
                reviews += find_all_reviews_of_a_product(product_link)
        else:
            break
            
        return reviews